In [1]:
%load_ext autoreload
%autoreload 2

In [32]:
import re
from itertools import chain

In [ ]:
def parse_operators(s):
    """
    Converts a string of the form "foo(bar(1), bar(2)) into a nested list [foo,[[bar,1],[bar,2]]]
    Used for converting z3 expression string into infix format
    """
    s = collapse_whitespace(s)
    

In [6]:
s = """And(And(Not(passenger-in-taxi(curly, t0)),
        Not(passenger-in-taxi(smoov, t0)),
        Not(passenger-in-taxi(littman, t0)),
        Not(passenger-in-taxi(isbell, t0))))"""

In [9]:
def collapse_whitespace(s):
    rex = re.compile(r'\s+')
    return rex.sub(' ', s)

In [10]:
collapse_whitespace(s)

'And(And(Not(passenger-in-taxi(curly, t0)), Not(passenger-in-taxi(smoov, t0)), Not(passenger-in-taxi(littman, t0)), Not(passenger-in-taxi(isbell, t0))))'

In [17]:
left_parens = [x.start() for x in re.finditer("\(",s)]
right_parens = [x.start() for x in re.finditer("\)",s)]

[3, 7, 11, 29, 54, 72, 97, 115, 142, 160]


In [ ]:
assert len(left_parens) == len(right_parens)


In [19]:
print([s[i] for i in left_parens])

['(', '(', '(', '(', '(', '(', '(', '(', '(', '(']


In [21]:
dum = [1,2,3]
dum.pop()
print(dum)

[1, 2]


In [28]:
def get_matching_parens(s):
    paren_pairs = []
    left_parens_stack = []
    for i, c in enumerate(s):
        if c == "(":
            left_parens_stack.append(i)
        if c == ")":
            if len(left_parens_stack) == 0:
                raise ValueError(f"Unbalanced Parens: {i}")
            paren_pairs.append((left_parens_stack.pop(), i))
    if len(left_parens_stack) > 0:
        raise ValueError(f"Extra left parens: {left_parens_stack}")
    return sorted(paren_pairs, key=lambda x: x[0])

In [ ]:
def slice_parens(s, paren_pairs)

In [26]:
?sorted

Signature: sorted(iterable, /, *, key=None, reverse=False)
Docstring:
Return a new list containing all items from the iterable in ascending order.

A custom key function can be supplied to customize the sort order, and the
reverse flag can be set to request the result in descending order.
Type:      builtin_function_or_method


In [43]:
s = collapse_whitespace(s); print(s)

And(And(Not(passenger-in-taxi(curly, t0)), Not(passenger-in-taxi(smoov, t0)), Not(passenger-in-taxi(littman, t0)), Not(passenger-in-taxi(isbell, t0))))


In [44]:
paren_pairs = get_matching_parens(s)

In [51]:
split_idx = sorted(list(chain(*paren_pairs)))
if 0 not in split_idx:
    split_idx = [-1] + split_idx

In [52]:
s_pieces = []
for i in range(len(split_idx) - 1):
    s_pieces.append(s[split_idx[i] + 1:split_idx[i+1]])

In [53]:
s_pieces

['And',
 'And',
 'Not',
 'passenger-in-taxi',
 'curly, t0',
 '',
 ', Not',
 'passenger-in-taxi',
 'smoov, t0',
 '',
 ', Not',
 'passenger-in-taxi',
 'littman, t0',
 '',
 ', Not',
 'passenger-in-taxi',
 'isbell, t0',
 '',
 '',
 '']

In [ ]:
def listify(s, paren_pairs):
    

In [54]:
# https://stackoverflow.com/a/23185606
def parse_nested(text, left=r'[(]', right=r'[)]', sep=r','):
    """ Based on https://stackoverflow.com/a/17141899/190597 (falsetru) """
    pat = r'({}|{}|{})'.format(left, right, sep)
    tokens = re.split(pat, text)    
    stack = [[]]
    for x in tokens:
        if not x or re.match(sep, x): continue
        if re.match(left, x):
            stack[-1].append([])
            stack.append(stack[-1][-1])
        elif re.match(right, x):
            stack.pop()
            if not stack:
                raise ValueError('error: opening bracket is missing')
        else:
            stack[-1].append(x)
    if len(stack) > 1:
        print(stack)
        raise ValueError('error: closing bracket is missing')
    return stack.pop()

In [57]:
l = parse_nested(s); l

['And',
 ['And',
  ['Not',
   ['passenger-in-taxi', ['curly', ' t0']],
   ' Not',
   ['passenger-in-taxi', ['smoov', ' t0']],
   ' Not',
   ['passenger-in-taxi', ['littman', ' t0']],
   ' Not',
   ['passenger-in-taxi', ['isbell', ' t0']]]]]

In [63]:
len(l)

2

In [58]:
print("~")

~


In [69]:
operator_symbols = {
    "And": "&",
    "Or": "|",
}
def nested_list2str(l):
    if isinstance(l, str):
        return l
    # elif list_is_flat(l):
    #     return ",".join(l)
    else:
        assert len(l) == 2, f"{len(l)}; {l}"
        operator, args = l
        args = [nested_list2str(x) for x in args]
        if operator == "Not":
            assert len(args) == 1, l
            return f"~{args[0]}"
        else:
            operator_symbol = operator_symbols[operator]
            return "(" + operator_symbol.join(args)



        

In [70]:
nested_list2str(l)

AssertionError: 8; ['Not', ['passenger-in-taxi', ['curly', ' t0']], ' Not', ['passenger-in-taxi', ['smoov', ' t0']], ' Not', ['passenger-in-taxi', ['littman', ' t0']], ' Not', ['passenger-in-taxi', ['isbell', ' t0']]]

In [64]:
l[0]

'And'

In [65]:
l[1]

['And',
 ['Not',
  ['passenger-in-taxi', ['curly', ' t0']],
  ' Not',
  ['passenger-in-taxi', ['smoov', ' t0']],
  ' Not',
  ['passenger-in-taxi', ['littman', ' t0']],
  ' Not',
  ['passenger-in-taxi', ['isbell', ' t0']]]]

In [74]:
l

['And',
 ['And',
  ['Not',
   ['passenger-in-taxi', ['curly', ' t0']],
   ' Not',
   ['passenger-in-taxi', ['smoov', ' t0']],
   ' Not',
   ['passenger-in-taxi', ['littman', ' t0']],
   ' Not',
   ['passenger-in-taxi', ['isbell', ' t0']]]]]

In [71]:
len(l)

2

In [73]:
len(l[1][0])

3